García Girón, Alejandro

ETL//Examen Final
MDS / CUNEF




COMENTARIOS INICIALES

La base de datos parseada manualmente (linea9,,,, 519, 540, 869, 1121, 1124, 1456, 1729, 1753, 1785, 1801, 1841, 1842, 2231, 2262, 2383, 2394, 2457,  sin_rvst y linea23 ,,,, 2238  sin_grj) la use en otro notebook de ensayo donde el parseado solo consistió en aplicar seaparador.

En este notebook no consigo fijar las funciones donde se reemplacen los huecos. (están definidas)

el parseado consistió en un split por coma y una trata de los valores de las columnas 0, 9 y 23.
los valores con la base parseada con notepad++ difieren en muy poco de la original.

La trata de duplicados no se habria llevado ya que son pocos (6), en la base parseada a mano sí se hizo. ya que se vio en excel y se manipuló en notepad++

el proceso ha sido llevado a cabo en Google Collaborate.





Preparamos el entorno

In [296]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [297]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [298]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Inicializar y cargar el contexto spark

In [299]:
#Carga de datos

casa = sc.textFile('bd_etl.csv')

In [300]:
#Comprobación de carga óptima de datos

casa.count()

2937

In [301]:
#Parseado de los datos y limpieza
#Separamos las bases de datos por p y  comas ';'

casa_alfa = casa.map(lambda x :x.split(";"))
casa_alfa.take(3)

[['Order',
  'MS Zoning',
  'Lot Frontage',
  'Lot Area',
  'Land Contour',
  'Land Slope',
  'Overall Qual',
  'Year Built',
  'Year Remod/Add',
  'Mas Vnr Type',
  'Exter Qual',
  'Bsmt Cond',
  'Total Bsmt SF',
  'Heating QC',
  'Central Air',
  'Full Bath',
  'Half Bath',
  'Bedroom AbvGr',
  'Kitchen AbvGr',
  'Kitchen Qual',
  'TotRms AbvGrd',
  'Garage Cars',
  'Garage Area',
  'Garage Cond',
  'Pool Area',
  'Pool QC',
  'Mo Sold',
  'Yr Sold',
  'SalePrice'],
 ['1',
  'RL',
  '141',
  '31770',
  'Lvl',
  'Gtl',
  '6',
  '1960',
  '1960',
  'Stone',
  'TA',
  'Gd',
  '1080',
  'Fa',
  'Y',
  '1',
  '0',
  '3',
  '1',
  'TA',
  '7',
  '2',
  '528',
  'TA',
  '0',
  'Sp',
  '5',
  '2010',
  '215000'],
 ['2',
  'RH',
  '80',
  '11622',
  'Lvl',
  'Gtl',
  '5',
  '1961',
  '1961',
  'None',
  'TA',
  'TA',
  '882',
  'TA',
  'Y',
  '1',
  '0',
  '2',
  '1',
  'TA',
  '5',
  '1',
  '730',
  'TA',
  '0',
  'Sp',
  '6',
  '2010',
  '105000']]

In [302]:
#Recuento de los datos

casa_alfa.count()

2937

In [303]:
#eliminamos encabezado que nos está dando problemas 
#cuando conteamos los valores de cada rdd se suma uno distinto que es el nombre de la variable
#por eso, la suprimimos.

encab = casa_alfa.first()
casa_alfa = casa_alfa.filter(lambda x : x != encab) 

#comprobamos, efectivamente:

casa_alfa.take(2)


[['1',
  'RL',
  '141',
  '31770',
  'Lvl',
  'Gtl',
  '6',
  '1960',
  '1960',
  'Stone',
  'TA',
  'Gd',
  '1080',
  'Fa',
  'Y',
  '1',
  '0',
  '3',
  '1',
  'TA',
  '7',
  '2',
  '528',
  'TA',
  '0',
  'Sp',
  '5',
  '2010',
  '215000'],
 ['2',
  'RH',
  '80',
  '11622',
  'Lvl',
  'Gtl',
  '5',
  '1961',
  '1961',
  'None',
  'TA',
  'TA',
  '882',
  'TA',
  'Y',
  '1',
  '0',
  '2',
  '1',
  'TA',
  '5',
  '1',
  '730',
  'TA',
  '0',
  'Sp',
  '6',
  '2010',
  '105000']]

In [304]:
#comenzamos el parseado 
#trata de valores anómalos en columnas categóricas, col 1, 9 y 23

casa_alfa.map(lambda x: x[1]).distinct().collect()

['RL', 'RH', 'C', 'Rl', 'FV', 'RM', 'I', 'A', 'rL']

In [305]:
def ParseaditoBombaAtomica (x):
    if x == 'Rl':
        xn = 'RL'
    elif x == 'rL':
        xn='RL'
    else:
        xn = x
    return (xn)

#esta función nos permite integrar los valores Rl y rL en el conjunto RL

In [306]:
casa_alfa.map(lambda x: ParseaditoBombaAtomica(x[1])).distinct().collect()

#Se comprueba que así sea y vemos que los datos Rl y rL se integran en RL

['RL', 'RH', 'C', 'FV', 'RM', 'I', 'A']

In [307]:
casa_alfa.map(lambda x: x [9]).distinct().collect()

#Analizamos la columna 9 que detectamos valores vacios

['None', '', 'CBlock', 'Stone', 'BrkFace', 'BrkCmn']

In [308]:
def ParseaditoBombaAtomica1 (x):
    if x == '':
        xn = 'sin_rvst'
    
    else:
        xn = x
    return (xn)

  #Creamos una función que a sustituya los huecos vacios

In [309]:
casa_alfa.map(lambda x: ParseaditoBombaAtomica1(x[9])).distinct().collect()

#Denominamos los huecos vacios como 'sin_rvst'

['None', 'sin_rvst', 'CBlock', 'Stone', 'BrkFace', 'BrkCmn']

In [310]:
#Analizamos la columna 23 que también tiene espacios vacios

casa_alfa.map(lambda x: x[23]).distinct().collect()

['Fa', 'Po', '', 'TA', 'Sg', 'Gd', 'Ex']

In [311]:
def ParseaditoBombaAtomica2 (x):
    if x == '':
        xn = 'sin_grj'
    
    else:
        xn = x
    return (xn)

 #Creamos una función que a sustituya los huecos vacios por 'sin_grj'

In [312]:
casa_alfa.map(lambda x: ParseaditoBombaAtomica2(x[23])).distinct().collect()

#Denominamos los huecos vacios como 'sin_grj'

['Fa', 'Po', 'sin_grj', 'TA', 'Sg', 'Gd', 'Ex']

Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?


In [313]:
print('Cuántas viviendas distintas encontramos en el dataset?.', 
casa_alfa.map(lambda x: x[0]).count(),
'¿Se repite alguna?',
casa_alfa.map(lambda x: x[0]).distinct().count(),
'hay 6, repetidas. ¿Tiene sentido que haya duplicadas? no tiene', 
'pero debido a algun error en  la recopilación hay 6 filas que se repiten. ¿Qué podemos hacer con las duplicadas? En este caso, eliminanrlas.')

Cuántas viviendas distintas encontramos en el dataset?. 2936 ¿Se repite alguna? 2930 hay 6, repetidas. ¿Tiene sentido que haya duplicadas? no tiene pero debido a algun error en  la recopilación hay 6 filas que se repiten. ¿Qué podemos hacer con las duplicadas? En este caso, eliminanrlas.


## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

In [314]:
zonaRL = casa_alfa.filter(lambda x: x[1] == 'RL')
zonaRM = casa_alfa.filter(lambda x: x[1] == 'RM')
zonaRH = casa_alfa.filter(lambda x: x[1] == 'RH')

In [315]:
#zona x[1], vamos a usar 3 zonas solo. de acuerdo con el enunciado
#precio x[28]

inmb_precio_RL = casa_alfa.filter(lambda x: x[1] == 'RL').map(lambda x: (x[0], x[28])) 
inmb_precio_RM = casa_alfa.filter(lambda x: x[1] == 'RM').map(lambda x: (x[0], x[28])) 
inmb_precio_RH = casa_alfa.filter(lambda x: x[1] == 'RH').map(lambda x: (x[0], x[28])) 

#Contamos los inmuebles de cada clase
inmb_precio_RL.count() + inmb_precio_RM.count() +inmb_precio_RH.count()


2760

In [316]:
inmb_precio_RL.count() 

2271

In [317]:
#funcion que pasa a float, ya que los numeros es necesario que esten en este formato para operar
def tofloat(x):
        if x in ["", " ","nan","NULL"]: 
            return 0.0
        else:
            return float(x)

In [318]:
print('El precio medio de la zona RL es,', inmb_precio_RL.map(lambda x: tofloat(x[1])).mean(), 'Dólares')

El precio medio de la zona RL es, 191131.80581241753 Dólares


In [319]:
print('El precio medio de la zona RM es,',inmb_precio_RM.map(lambda x: tofloat(x[1])).mean() , 'Dólares')

El precio medio de la zona RM es, 126781.39393939397 Dólares


In [320]:
print('El precio medio de la zona RH es,',inmb_precio_RH.map(lambda x: tofloat(x[1])).mean() , 'Dólares')

El precio medio de la zona RH es, 136419.77777777778 Dólares


##Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 

In [321]:
casa_alfa.map(lambda x: x[7]).distinct().collect()

#el primer registro es de 1872, por lo que trabajaremos con 15 decadas

['1995',
 '1993',
 '1990',
 '2003',
 '2010',
 '1951',
 '1978',
 '1977',
 '1970',
 '2007',
 '2004',
 '2002',
 '1994',
 '1980',
 '1979',
 '1920',
 '1962',
 '1964',
 '1949',
 '1940',
 '1954',
 '1957',
 '1956',
 '1948',
 '1927',
 '1938',
 '1928',
 '1890',
 '1925',
 '1939',
 '1936',
 '1921',
 '1907',
 '1969',
 '1947',
 '1941',
 '1896',
 '1991',
 '1981',
 '1880',
 '1937',
 '1934',
 '1982',
 '1983',
 '1918',
 '1872',
 '1911',
 '1879',
 '1901',
 '1913',
 '1960',
 '1961',
 '1958',
 '1968',
 '1997',
 '1998',
 '2001',
 '1992',
 '1999',
 '1985',
 '1988',
 '1974',
 '2000',
 '1971',
 '1975',
 '2009',
 '2005',
 '2006',
 '1996',
 '2008',
 '1984',
 '1965',
 '1967',
 '1963',
 '1976',
 '1972',
 '1966',
 '1959',
 '1950',
 '1952',
 '1955',
 '1953',
 '1900',
 '1910',
 '1915',
 '1945',
 '1929',
 '1923',
 '1885',
 '1922',
 '1942',
 '1930',
 '1912',
 '1917',
 '1875',
 '1946',
 '1987',
 '1924',
 '1914',
 '1931',
 '1919',
 '1989',
 '1973',
 '1986',
 '1916',
 '1926',
 '1935',
 '1892',
 '1898',
 '1882',
 '1902',
 

In [322]:
decada1 = casa_alfa.filter(lambda x: x[7] < '1880').map(lambda x: (x[0], x[7], x[12], x[13]))
decada1.collect()
print('Para la década de 1870 los pies cuadrados promedio del sótano son', decada1.map(lambda x: tofloat(x[2])).mean())

Para la década de 1870 los pies cuadrados promedio del sótano son 761.0


In [323]:
decada2 = casa_alfa.filter(lambda x:'1880'<= x[7] < '1890').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1880 los pies cuadrados promedio del sótano son', decada2.map(lambda x: tofloat(x[2])).mean())

Para la década de 1880 los pies cuadrados promedio del sótano son 836.0


In [324]:
decada3 = casa_alfa.filter(lambda x:'1890'<= x[7] < '1900').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1890 los pies cuadrados promedio del sótano son', decada3.map(lambda x: tofloat(x[2])).mean())

Para la década de 1890 los pies cuadrados promedio del sótano son 834.2


In [241]:
decada4 = casa_alfa.filter(lambda x:'1900'<= x[7] < '1910').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1900 los pies cuadrados promedio del sótano son', decada4.map(lambda x: tofloat(x[2])).mean())

Para la década de 1900 los pies cuadrados promedio del sótano son 710.55


In [242]:
decada5 = casa_alfa.filter(lambda x:'1910'<= x[7] < '1920').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1910 los pies cuadrados promedio del sótano son', decada5.map(lambda x: tofloat(x[2])).mean())

Para la década de 1910 los pies cuadrados promedio del sótano son 778.5272727272727


In [243]:
decada5 = casa_alfa.filter(lambda x:'1920'<= x[7] < '1930').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1920 los pies cuadrados promedio del sótano son', decada5.map(lambda x: tofloat(x[2])).mean())

Para la década de 1920 los pies cuadrados promedio del sótano son 828.1428571428571


In [244]:
decada6 = casa_alfa.filter(lambda x:'1930'<= x[7] < '1940').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1930 los pies cuadrados promedio del sótano son', decada5.map(lambda x: tofloat(x[2])).mean())

Para la década de 1930 los pies cuadrados promedio del sótano son 828.1428571428571


In [245]:
decada7 = casa_alfa.filter(lambda x:'1940'<= x[7] < '1950').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1940 los pies cuadrados promedio del sótano son', decada7.map(lambda x: tofloat(x[2])).mean())

Para la década de 1940 los pies cuadrados promedio del sótano son 700.7417218543046


In [246]:
decada8 = casa_alfa.filter(lambda x:'1950'<= x[7] < '1960').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1950 los pies cuadrados promedio del sótano son', decada8.map(lambda x: tofloat(x[2])).mean())

Para la década de 1950 los pies cuadrados promedio del sótano son 960.7411764705882


In [247]:
decada9 = casa_alfa.filter(lambda x:'1960'<= x[7] < '1970').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1960 los pies cuadrados promedio del sótano son', decada9.map(lambda x: tofloat(x[2])).mean())

Para la década de 1960 los pies cuadrados promedio del sótano son 1082.6722689075627


In [248]:
decada10 = casa_alfa.filter(lambda x:'1970'<= x[7] < '1980').map(lambda x: (x[0], x[7], x[12], x[13], x[13]))
print('Para la década de 1970 los pies cuadrados promedio del sótano son', decada10.map(lambda x: tofloat(x[2])).mean())

Para la década de 1970 los pies cuadrados promedio del sótano son 953.9178082191784


In [249]:
decada11 = casa_alfa.filter(lambda x:'1980'<= x[7] < '1990').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1980 los pies cuadrados promedio del sótano son', decada11.map(lambda x: tofloat(x[2])).mean())

Para la década de 1980 los pies cuadrados promedio del sótano son 1086.0000000000002


In [250]:
decada12 = casa_alfa.filter(lambda x:'1990'<= x[7] < '2000').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década de 1990 los pies cuadrados promedio del sótano son', decada12.map(lambda x: tofloat(x[2])).mean())

Para la década de 1990 los pies cuadrados promedio del sótano son 1124.0863095238103


In [251]:
decada13 = casa_alfa.filter(lambda x:'2000'<= x[7] < '2010').map(lambda x: (x[0], x[7], x[12], x[13]))
print('Para la década del 2000 los pies cuadrados promedio del sótano son', decada13.map(lambda x: tofloat(x[2])).mean())

Para la década del 2000 los pies cuadrados promedio del sótano son 1311.3997445721573


## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

In [325]:
#Apliamos un filtro por Ex y conteamos

acond1 = decada1.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond2= decada2.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond3 = decada3.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond4 = decada4.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond5 = decada5.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond6 = decada6.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond7 = decada7.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond8 = decada8.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond9 = decada9.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond10 = decada10.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond11 = decada11.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond12 = decada12.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))
acond13 = decada13.filter(lambda x: x[3] == 'Ex').map(lambda x: (x[0], x[3]))

In [326]:
print(
acond1.count(), 
acond2.count(), 
acond3.count(), 
acond4.count(), 
acond5.count(), 
acond6.count(), 
acond7.count(), 
acond8.count(), 
acond9.count(), 
acond10.count(), 
acond11.count(), 
acond12.count(), 
acond13.count(), 'visto el número de viviendas con calefacción excelente la mejor década en cifras absolutas es la última' )

0 2 5 13 52 41 56 98 88 68 28 251 755 visto el número de viviendas con calefacción excelente la mejor década en cifras absolutas es la última


## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

In [327]:
#Aplicar Reduce By Key nos hace ahorrar código vemos que en otras celdas no se ha aplicado


año2009 = casa_alfa.filter(lambda x: x[27] == '2009').map(lambda x: (x[0], (tofloat(x[28])/tofloat(x[3])))).reduceByKey(lambda a,b:a + b)
año2009.collect()

[('342', 9.571033210332104),
 ('344', 24.89177489177489),
 ('345', 18.61675999607034),
 ('346', 22.89687924016282),
 ('347', 22.5),
 ('348', 25.34918076819769),
 ('351', 49.0819209039548),
 ('353', 29.88893048795707),
 ('354', 19.892412865628152),
 ('356', 25.466666666666665),
 ('359', 20.597069180961036),
 ('361', 18.47239518472395),
 ('365', 15.23078221247152),
 ('370', 15.546218487394958),
 ('372', 10.62019597488705),
 ('375', 11.622641509433961),
 ('376', 18.301610541727673),
 ('378', 13.28374070138151),
 ('380', 28.958333333333332),
 ('383', 19.167313664596275),
 ('385', 32.912918540729635),
 ('386', 29.333333333333332),
 ('387', 27.744982290436838),
 ('389', 16.558185404339252),
 ('390', 14.298642533936652),
 ('408', 71.13095238095238),
 ('410', 63.258232235701904),
 ('411', 65.13157894736842),
 ('412', 53.94736842105263),
 ('413', 63.69150779896014),
 ('414', 58.70583226905066),
 ('417', 58.3756345177665),
 ('419', 13.986013986013987),
 ('422', 32.40513431998317),
 ('426', 18.01

In [328]:
#De los registros analizados el que mayor precio tiene por metro cuadrado es el 568 

año2009.max(lambda x: x[1])

('566', 97.48263888888889)

In [329]:
datos_de_max = año2009.sortBy(lambda x: tofloat(x[1]), ascending = False) 
print('Los registros mas caros ordenados de manera descendente son', datos_de_max.take(10))

Los registros mas caros ordenados de manera descendente son [('566', 97.48263888888889), ('936', 95.38461538461539), ('934', 92.06989247311827), ('464', 76.27865961199295), ('935', 75.06203473945409), ('408', 71.13095238095238), ('407', 70.23809523809524), ('405', 66.66666666666667), ('933', 66.35891517599538), ('403', 66.07142857142857)]


## Media anual por zonas del precio de venta y metros cuadrados

Aquí empleamos el reducebykey para hacerlo por claves en funcion del numero que queramos recuperar del rdd ya que ahorramos muchas lineas de código

In [330]:
pvZonaRL = zonaRL.map(lambda x: (x[7], [tofloat(x[28]), tofloat(x[3]), 1]))
pvZonaRL = pmRL.reduceByKey(lambda x,y: [x[0]+y[0], x[1]+y[1], x[2]+y[2]]).map(lambda x: (x[0], x[1][0]/x[1][2], x[1][1]/x[1][2]))
pvZonaRL.take(10)

[('1978', 112000.0, 8930.0),
 ('1920', 113161.29032258065, 6772.129032258064),
 ('1927', 123333.33333333333, 7197.166666666667),
 ('1957', 136050.0, 7068.0),
 ('1940', 118778.54545454546, 6859.363636363636),
 ('1938', 125466.66666666667, 6520.0),
 ('1948', 132111.11111111112, 8596.666666666666),
 ('1928', 138633.33333333334, 8120.0),
 ('1890', 160680.0, 9066.8),
 ('1925', 113547.22222222222, 7013.555555555556)]

In [331]:
pvZonaRM = zonaRM.map(lambda x: (x[7], [tofloat(x[28]), tofloat(x[3]), 1]))
pvZonaRM = pvZonaRM.reduceByKey(lambda x,y: [x[0]+y[0], x[1]+y[1], x[2]+y[2]]).map(lambda x: (x[0], x[1][0]/x[1][2], x[1][1]/x[1][2]))
pvZonaRM.take(10)

[('1978', 112000.0, 8930.0),
 ('1920', 113161.29032258065, 6772.129032258064),
 ('1927', 123333.33333333333, 7197.166666666667),
 ('1957', 136050.0, 7068.0),
 ('1940', 118778.54545454546, 6859.363636363636),
 ('1938', 125466.66666666667, 6520.0),
 ('1948', 132111.11111111112, 8596.666666666666),
 ('1928', 138633.33333333334, 8120.0),
 ('1890', 160680.0, 9066.8),
 ('1925', 113547.22222222222, 7013.555555555556)]

In [332]:
pvZonaRH = zonaRH.map(lambda x: (x[7], [tofloat(x[28]), tofloat(x[3]), 1]))
pvZonaRH = pvZonaRH.reduceByKey(lambda x,y: [x[0]+y[0], x[1]+y[1], x[2]+y[2]]).map(lambda x: (x[0], x[1][0]/x[1][2], x[1][1]/x[1][2]))
pvZonaRH.take(10)

[('1920', 67500.0, 9800.0),
 ('1928', 76000.0, 9000.0),
 ('1925', 121800.0, 9392.0),
 ('1938', 160000.0, 6300.0),
 ('1911', 130000.0, 8525.0),
 ('1977', 144000.0, 8400.0),
 ('1949', 145000.0, 6270.0),
 ('2002', 190000.0, 8773.0),
 ('1993', 137000.0, 3612.0),
 ('1961', 105000.0, 11622.0)]

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa

In [333]:
ladrillo = casa_alfa.filter(lambda x: x[9] == 'BrkCmn').map(lambda x: (x[0], x[9], x[28]))
Stones  = casa_alfa.filter(lambda x: x[9] == 'Stone').map(lambda x: (x[0], x[9], x[28]))

In [334]:
precioladrillo = ladrillo.map(lambda x: tofloat(x[2])).sum()
print('el precio medio de las viviendas de ladrillo es de', precioladrillo)

el precio medio de las viviendas de ladrillo es de 3504975.0


In [335]:
preciostone = Stones.map(lambda x: tofloat(x[2])).sum()
print('el precio medio de las viviendas de piedra es de', preciostone, 'la diferencia entre estas últimas es considerable ya que el monto difiere en más de 60 millones de Euros')

el precio medio de las viviendas de piedra es de 65196277.0 la diferencia entre estas últimas es considerable ya que el monto difiere en más de 60 millones de Euros


In [ ]:
#la diferencia proviene de el volumen de venta 25 de ladrillo frente a 250 de piedra, aun así el precio medio de venta de ladrillo es de 140 mil Euros
#frente a 260 mil Euros

## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

In [336]:
dos_masdedos = casa_alfa.filter(lambda x: x[18] == '2' and x[21] > '2').map(lambda x: (x[0], x[18], x[21], x[28]))
print('El precio medio de las casa con dos cocinas y mas de dos plazas de garaje es de',dos_masdedos.map(lambda x: tofloat(x[3])).mean(), 'Euros')


El precio medio de las casa con dos cocinas y mas de dos plazas de garaje es de 180611.11111111112 Euros


In [337]:
uno_y_uno = casa_alfa.filter(lambda x: x[18] == '1' and x[21] == '1').map(lambda x: (x[0], x[18], x[21], x[28]))
print('El precio medio de las casa con una cocina y una plaza de garaje es de', uno_y_uno.map(lambda x: tofloat(x[3])).mean(), 'Euros')

El precio medio de las casa con una cocina y una plaza de garaje es de 127367.04161248378 Euros
